<a href="https://colab.research.google.com/github/Bhekmuzi/water-usage-norm/blob/main/src/data_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip freeze > requirements.txt

In [ ]:
!pip install influxdb

In [ ]:
!pip install pandas pymongo

In [4]:
import numpy as np
from sklearn.cluster import KMeans
# pip install influxdb
# !python --version
import numpy as np
import pandas as pd
import datetime
import time
from influxdb import InfluxDBClient
from pymongo import MongoClient
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
# from scipy.stats import entropy

# Provide the IP address, username, password, database name, RFC3339 standard time format, and create a connection client for the 'db0' database
client = InfluxDBClient('59.120.114.133', 8086, 'telegraf', 'telegraf', 'db0', 'rfc3339', timeout=10)

In [5]:
# Function to calculate start and end times for the previous day
def calculate_previous_day_times():
    current_datetime = datetime.now()
    start_of_previous_day = current_datetime - timedelta(days=1)
    start_of_previous_day = start_of_previous_day.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_previous_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(microseconds=1)

    return start_of_previous_day, end_of_previous_day

In [6]:
# InfluxDB query
start_time, end_time = calculate_previous_day_times()

sql_string = f'SELECT DISTINCT("value") AS value FROM mbMQTT6 WHERE "topic" = \'mbMQTT2/home2127/C2BDF8/TH20\' AND time >= \'{start_time.strftime("%Y-%m-%dT%H:%M:%SZ")}\' AND time <= \'{end_time.strftime("%Y-%m-%dT%H:%M:%SZ")}\' GROUP BY time(10s) FILL(previous) ORDER BY time ASC TZ(\'Asia/Taipei\');'

result = client.query(sql_string) #

In [7]:

homes_water_data = {
    'home2127':pd.DataFrame(result['mbMQTT6']),
    'home2128':pd.DataFrame(result['mbMQTT6'])
}

In [8]:
# Function to fill missing values for a given home
def fill_missing_values(home_data):
    home_data['time'] = pd.to_datetime(home_data['time'])
    home_data.set_index('time', inplace=True)
    expected_time_intervals = pd.date_range(start=home_data.index.min(), end=home_data.index.max(), freq='10S')
    home_data = home_data.reindex(expected_time_intervals)
    home_data['value'] = home_data['value'].fillna(method='pad')
    home_data.reset_index(inplace=True)
    return home_data

In [9]:
def process_home_data(home_water_df):
    # Step 1: Fill missing values
    filled_home_data = fill_missing_values(home_water_df)

    # Step 2: Convert 'value' column to numeric and perform division and multiplication
    filled_home_data['value'] = pd.to_numeric(filled_home_data['value'], errors='coerce')
    filled_home_data['value'] = filled_home_data['value'] / 100000 * 1000

    # Step 3: Calculate the difference between 'value' column
    filled_home_data['volume'] = filled_home_data['value'].diff()

    # Step 4: Replace NaN with 0 in the 'volume' column
    filled_home_data['volume'] = filled_home_data['volume'].fillna(0)

    # # Step 5: Filter values less than 0.2 in the 'volume' column
    # filtered_home_data = filled_home_data[filled_home_data['volume'] < 0.2]
    # Step 5: Filter values less than 0.2 in the 'volume' column and set them to 0
    filled_home_data['volume'] = filled_home_data['volume'].apply(lambda x: 0 if x < 0.2 else x)

    # Step 6: Rename 'index' column to 'time'
    filled_home_data.reset_index(drop=True, inplace=True)
    filled_home_data.rename(columns={'index': 'time'}, inplace=True)

    # return filtered_home_data
    return filled_home_data



In [10]:
# Example usage for each home
homes_data_processed = {}

for home_name, home_water_df in homes_water_data.items():
    processed_data = process_home_data(home_water_df)
    homes_data_processed[home_name] = processed_data
    # print(f"\nProcessed Data for {home_name}:\n{processed_data}")


In [11]:

import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np
from scipy.stats import entropy

def process_events(df):
    start_time = None
    end_time = None
    consecutive_zeros = 0
    total_time = timedelta()
    total_vol = 0
    num_records = 0

     # Create a new DataFrame to store event information
    event_df = pd.DataFrame(columns=['Start Time', 'End Time'])

    for index, row in df.iterrows():
        time = row['time']
        vol = max(0, float(row['volume']))

        if vol != 0:
            if start_time is None:
                start_time = time - timedelta(seconds=60)
            consecutive_zeros = 0
        else:
            consecutive_zeros += 1
            if start_time is not None and consecutive_zeros == 6:
                end_time = time
                # print(f"Event start_time: {start_time}, end_time: {end_time}")

                # Filter volume values between start_time and end_time
                event_data = df[(df['time'] >= start_time) & (df['time'] <= end_time)]

                # Calculate total time, total volume, and coefficient of variation
                time1 = start_time + timedelta(seconds=60)
                time2 = end_time - timedelta(seconds=60)
                event_data1 = df[(df['time'] >= time1) & (df['time'] <= time2)]

                event_df = pd.concat([event_df, pd.DataFrame({
                    'Start Time': [time1],
                    'End Time': [time2],
                })], ignore_index=True)

                start_time = None  # Reset start_time for the next event

    if event_df.empty:
        print("No events detected.")
        return pd.DataFrame()  # Return an empty DataFrame if no events are detected

    return event_df

In [12]:
def process_and_mark_usage(home_df):
    # Process events
    event_df = process_events(home_df)

    # Calculate the duration of each event
    event_df['Duration'] = (event_df['End Time'] - event_df['Start Time']).dt.total_seconds()

    # Filter out events with zero duration
    event_df = event_df[event_df['Duration'] != 0]

    # Filter events with duration less than 10 seconds
    event_atleast_10_df = event_df[event_df['Duration'] >= 10]

    # Initialize 'usage' column in home_df
    home_df['usage'] = 0

    # Iterate through rows in event_atleast_10_df and mark corresponding rows in home_df as 1
    for index, row in event_atleast_10_df.iterrows():
        mask = (home_df['time'] >= row['Start Time']) & (home_df['time'] <= row['End Time'])
        home_df.loc[mask, 'usage'] = 1

    return home_df


In [21]:
import pandas as pd
from pymongo import MongoClient
import json

def process_resample_insert(home_df, home_id):
    # Step 1: Process and mark usage
    processed_home_df = process_and_mark_usage(home_df)
    processed_home_df['time'] = pd.to_datetime(processed_home_df['time'])

    # Step 2: Resample to 15-minute intervals and take max values within each interval
    resampled_df = processed_home_df.set_index('time').resample('15T').max().reset_index()

    # Extract the date and add it as a new column
    resampled_df['date'] = resampled_df['time'].dt.date

    # Drop the 'value' column
    resampled_df = resampled_df.drop(columns=['value', 'time', 'volume'])

    # Calculate active_score
    active_score = round(resampled_df["usage"].sum() / 96 * 100, 3)

    # Group by 'date' and aggregate 'usage' column as a list
    grouped_df = resampled_df.groupby('date')['usage'].apply(list).reset_index()

    # Group by 'date' and aggregate 'usage' column as a list
    grouped_df = resampled_df.groupby('date')['usage'].apply(list).reset_index()

    # Convert DataFrame to a dictionary
    result_dict = {}

    # load date, homeid, usage, and active_score into a json variable result_json

    for index, row in grouped_df.iterrows():
        # result_dict[row['date']] = {
        result_dict = {
            'date': row['date'],
            'homeID': home_id,
            'usage': row['usage'],
            'active_score': active_score
        }

    return result_dict

# Example usage of the function
resampled_home_data = {}

for home_name, filtered_renamed_home_data in homes_data_processed.items():
    resampled_data = process_resample_insert(filtered_renamed_home_data, home_name)
    resampled_home_data[home_name] = resampled_data
    print(f"\nResampled Data for {home_name}:\n{resampled_data}")


Resampled Data for home2127:
{'date': datetime.date(2024, 1, 14), 'homeID': 'home2127', 'usage': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0], 'active_score': 18.75}

Resampled Data for home2128:
{'date': datetime.date(2024, 1, 14), 'homeID': 'home2128', 'usage': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0], 'active_score': 18.75}


In [23]:
import pandas as pd
from pymongo import MongoClient
import json

def process_resample_insert(home_df, home_id):
    # Step 1: Process and mark usage
    processed_home_df = process_and_mark_usage(home_df)
    processed_home_df['time'] = pd.to_datetime(processed_home_df['time'])

    # Step 2: Resample to 15-minute intervals and take max values within each interval
    resampled_df = processed_home_df.set_index('time').resample('15T').max().reset_index()

    # Extract the date and add it as a new column
    resampled_df['date'] = resampled_df['time'].dt.date

    # Drop the 'value' column
    resampled_df = resampled_df.drop(columns=['value', 'time', 'volume'])

    # Calculate active_score
    active_score = round(resampled_df["usage"].sum() / 96 * 100, 3)

    # Group by 'date' and aggregate 'usage' column as a list
    grouped_df = resampled_df.groupby('date')['usage'].apply(list).reset_index()

    # Convert DataFrame to a dictionary
    result_dict = {
        'date': grouped_df['date'].tolist(),
        'homeID': home_id,
        'usage': grouped_df['usage'].tolist(),
        'active_score': [active_score] * len(grouped_df)
    }

    return result_dict

# Example usage of the function
resampled_home_data = {}

for home_name, filtered_renamed_home_data in homes_data_processed.items():
    resampled_data = process_resample_insert(filtered_renamed_home_data, home_name)
    resampled_home_data[home_name] = resampled_data
    print(f"\nResampled Data for {home_name}:\n{resampled_data}")



Resampled Data for home2127:
{'date': [datetime.date(2024, 1, 14)], 'homeID': 'home2127', 'usage': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0]], 'active_score': [18.75]}

Resampled Data for home2128:
{'date': [datetime.date(2024, 1, 14)], 'homeID': 'home2128', 'usage': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0]], 'active_score': [18.75]}


In [ ]:
import json

# Replace 'your_database' and 'your_collection' with your actual database and collection names
client = MongoClient('mongodb://34.81.144.96:27017/')
db = client['Taipower']
collection = db['SmartWaterMeterActiveHistory']

# json.loads(resampled_home_data)


# Insert the resampled_home_data into the MongoDB collection
for home_name, resampled_data in resampled_home_data.items():
    for date, data in resampled_data.items():
        collection.insert_one(data)

# Close the MongoDB connection
client.close()

In [25]:
# Example usage of the function
resampled_home_data = {}

for home_name, filtered_renamed_home_data in homes_data_processed.items():
    resampled_data = process_resample_insert(filtered_renamed_home_data, home_name)
    resampled_home_data[home_name] = resampled_data

    # Insert the data into MongoDB
    collection.insert_one(resampled_data)

    print(f"\nResampled Data for {home_name}:\n{resampled_data}")

ServerSelectionTimeoutError: 34.81.144.96:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65a49c92844a5946ca7c1766, topology_type: Unknown, servers: [<ServerDescription ('34.81.144.96', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('34.81.144.96:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# Query data from mongodb

# print(db.list_collection_names())
# client.close()
# Specify the homeid you want to query
homeid = "Home2127"

# Calculate the date of the previous day
end_date = datetime.now()
start_date  = (datetime.now() - timedelta(days=7))

# Query for documents with the specified homeid
query = {
    "home_id": homeid,
    "date": {"$gte": start_date.strftime('%Y-%m-%d'),
             "$lte": end_date.strftime('%Y-%m-%d')}
}
# query = {"home_id": homeid, "date": previous_day}
result = collection.find(query)

# Print the results
for document in result:
    print(document)

# Close the connection
client.close()

In [ ]:
# Convert dictionary back to usable data


In [ ]:
# Calculate active scores, Norms, and correlation coefficient

import pandas as pd

def calculate_metrics(input_df):
    # Sum the columns
    norm_sum = input_df.sum()

    # Calculate active score
    active_score = input_df.sum() / (len(input_df) * len(input_df.columns)) * 100

    # Sort DataFrame in ascending order by the first column
    sorted_df = input_df.sort_values(by=input_df.columns[0], ascending=True)

    # Find the index to split the DataFrame into two halves
    split_index = len(sorted_df) // 2

    # Split the DataFrame into the first and second halves
    first_half = sorted_df.iloc[:split_index, :]
    second_half = sorted_df.iloc[split_index:, :]

    # Calculate the averages for each half
    low_norm = first_half.mean()
    high_norm = second_half.mean()

    # Calculate overall norm
    overall_norm = sorted_df.mean()

    return {
        'active_score': active_score,
        'low_norm': low_norm,
        'norm': overall_norm,
        'high_norm': high_norm
    }

# Example usage
# Assuming 'two_week_norm' is your DataFrame
result = calculate_metrics(two_week_norm)

# Print the result
print(result)


In [ ]:
import json
age = [30, 20, 52, 85, 6]
# Simulating receiving JSON data from Golang
golang_json_data = '{"name": "John Doe", "age": ' + json.dumps(age) + ', "city": "Example City", "email": "john@example.com"}'

# Deserialize JSON data in Python
python_data = json.loads(golang_json_data)

print("Python Data:", python_data)
